### Ambiente

In [39]:
# Bibliotecas
import pandas as pd 
import mysql.connector as conn

import os
from dotenv import load_dotenv
load_dotenv()

In [40]:
# Parametros opcionais
pd.set_option('display.max_columns', None)

In [41]:
#> Parametros de Conexão com Banco de Dados
mydb = \
conn.connect(
              host =      os.getenv('HOST_DB'),
              user =      os.getenv('USER_DB'),
              password =  os.getenv('PASS_DB'),
              database =  os.getenv('DATA_DB')
            )

### Projeto

##### Transformacao dos dados

In [42]:
# Leitura dos arquivos
compras = pd.read_csv('../HubFinanceiro/Data/compras.csv',sep=',',encoding='utf-8')
fatura = pd.read_csv('../HubFinanceiro/Data/fatura.csv',sep=',',encoding='utf-8')

Compras

In [43]:
# Renomeando colunas
compras.columns = ['ITEM','TPO_PAGAMENTO','VALOR','DAT_TRANSACAO','ORIGEM','CATEGORIA',\
                   'VAL_IOF','CONTA','ID_COMPRA','TOKEN','LINK','STATUS', 'CARTAO_FISICO',\
                   'API_ID','TAGS', 'QTD_PARCELAS', 'VAL_PARCELAS','MOEDA','VAL_MOEDA','VAL_USD',\
                   'VAL_PRECISO','VAL_PRECISO_USD','VAL_CAMBIO','LATITUDE','LONGITUDE']

In [45]:
# Removendo colunas desnecessárias
compras = compras.drop(columns=['TPO_PAGAMENTO','ORIGEM','VAL_IOF','CONTA','LINK','STATUS','API_ID'\
                               ,'MOEDA','VAL_MOEDA','VAL_USD','VAL_PRECISO','VAL_PRECISO_USD','VAL_CAMBIO'])

In [46]:
# Preenchendo valores nulos
compras['TOKEN']        = compras['TOKEN'].fillna(False)
compras['QTD_PARCELAS'] = compras['QTD_PARCELAS'].fillna(0)
compras['VAL_PARCELAS'] = compras['VAL_PARCELAS'].fillna(0)
compras['LATITUDE']     = compras['LATITUDE'].fillna(0)
compras['LONGITUDE']    = compras['LONGITUDE'].fillna(0)
compras['TAGS']         = compras['TAGS'].fillna('-')

In [47]:
# Padronizando data
compras['DAT_TRANSACAO'] = pd.to_datetime(compras['DAT_TRANSACAO']) 
compras['DAT_TRANSACAO'] = compras['DAT_TRANSACAO'].dt.tz_localize(None)

In [48]:
# Adicionando casa decimal no campo de VALOR 
compras['VALOR']        = compras.VALOR.astype(str)
compras.VALOR           = compras.VALOR.str.slice(stop=-2)\
                          + '.'\
                          + compras.VALOR.str.slice(start=-2)

In [49]:
# Adicionando casa decimal no campo de VAL_PARCELAS 
compras['VAL_PARCELAS'] = compras.VAL_PARCELAS.astype(int)
compras['VAL_PARCELAS'] = compras.VAL_PARCELAS.astype(str)
compras.VAL_PARCELAS = compras.VAL_PARCELAS.str.slice(stop=-2)\
                       + '.'\
                       + compras.VAL_PARCELAS.str.slice(start=-2)

In [ ]:
# Padronizando categorias
compras['CATEGORIA']\
= compras.CATEGORIA.map({'casa'          : 'Alimentacao',\
                         'educação'      : 'Educacao',\
                         'eletrônicos'   : 'Eletronicos',\
                         'supermecado'   : 'Alimentacao',\
                         'restaurante'   : 'Alimentacao',\
                         'saúde'         : 'Saude',\
                         'serviços'      : 'Servicos',\
                         'transporte'    : 'Transporte',\
                         'vestuário'     : 'Vestuario',\
                         'lazer'         : 'Lazer',\
                         'viagem'        : 'Lazer'
                         })

In [51]:
# Definindo tipo de dado para cada campo
compras['ITEM']             = compras.ITEM.astype(str)
compras['VALOR']            = compras.VALOR.astype(float)
compras['DAT_TRANSACAO']    = compras.DAT_TRANSACAO.astype(str)
compras['CATEGORIA']        = compras.CATEGORIA.astype(str)
compras['ID_COMPRA']        = compras.ID_COMPRA.astype(str)
compras['TOKEN']            = compras.TOKEN.astype(str)
compras['CARTAO_FISICO']    = compras.CARTAO_FISICO.astype(str)
compras['QTD_PARCELAS']     = compras.QTD_PARCELAS.astype(int)
compras['VAL_PARCELAS']     = compras.VAL_PARCELAS.astype(float)
compras['LATITUDE']         = compras.LATITUDE.astype(float)
compras['LONGITUDE']        = compras.LONGITUDE.astype(float)
compras['TAGS']             = compras.TAGS.astype(str)

Fatura

In [52]:
# Renomeando colunas
fatura.columns = ['SIT_FATURA','DAT_VENCIMENTO','DAT_FECHAMENTO','VAL_ANTERIOR',\
                  'DAT_VENCIMENTO_EFETIVO','VAL_GASTO','VAL_TOTAL','VAL_TX_JUROS',\
                  'VAL_JUROS','VAL_ACUMULADO','VAL_PAGO','VAL_PAGAMENTO_MIN', 'DAT_ABERTURA',\
                  'VAL_TX_JUROS_ANTERIOR', 'VAL_JUROS_ANTERIOR','API_ID','COD_BARRAS_BOLETO',\
                  'COD_EMAIL_BOLETO','ID_FATURA','SALDO_RESTANTE','SALDO_RESTANTE_MIN']

In [53]:
# Removendo colunas desnecessárias
fatura = fatura.drop(columns=['VAL_ANTERIOR','VAL_GASTO','VAL_ACUMULADO','VAL_PAGO','VAL_PAGAMENTO_MIN',\
                              'VAL_TX_JUROS_ANTERIOR','VAL_JUROS_ANTERIOR','API_ID','COD_BARRAS_BOLETO',\
                              'COD_EMAIL_BOLETO','SALDO_RESTANTE','SALDO_RESTANTE_MIN'])

In [54]:
# Preenchendo valores nulos
fatura['VAL_JUROS']    = fatura['VAL_JUROS'].fillna(0)
fatura['ID_FATURA']    = fatura['ID_FATURA'].fillna('-')

In [55]:
# Traduzindo campo SIT_FATURA
fatura['SIT_FATURA']\
= fatura.SIT_FATURA.map({'open'    : 'Aberta',\
                         'overdue' : 'Fechada',\
                         'future'  : 'Futura'})

In [56]:
# Adicionando casa decimal no campo de VAL_TOTAL 
fatura['VAL_TOTAL']        = fatura.VAL_TOTAL.astype(str)
fatura.VAL_TOTAL           = fatura.VAL_TOTAL.str.slice(stop=-2)\
                             + '.'\
                             + fatura.VAL_TOTAL.str.slice(start=-2)

##### Armazenamento dos dados

In [59]:
# Conectando ao banco de dados
mycursor = mydb.cursor()

In [60]:
# Inserindo informação no Banco de dados

# Compras
for row in compras.iterrows():              # Percorrendo todas as linhas do dataframe

    sql = "INSERT INTO NU_CARTAO \
           (ITEM, VALOR, DAT_TRANSACAO, CATEGORIA, ID_COMPRA, TOKEN, CARTAO_FISICO,disc TAGS, QTD_PARCELAS, VAL_PARCELAS, LATITUDE, LONGITUDE)\
           VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    
    val =  list(row[1])                     # Atribuindo cada linha a uma lista 
    mycursor.executemany(sql, (val,))       # Executando comando
    
mydb.commit()
print(compras.shape[0], "Registros Inseridos.")

# Fatura
for row in fatura.iterrows():
    sql = "INSERT INTO NU_FATURA \
           (SIT_FATURA, DAT_VENCIMENTO, DAT_FECHAMENTO, DAT_VENCIMENTO_EFETIVO, VAL_TOTAL, VAL_TX_JUROS, VAL_JUROS, DAT_ABERTURA, ID_FATURA)\
           VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val =  list(row[1])
    mycursor.executemany(sql, (val,))

mydb.commit()
print(fatura.shape[0], "registros inseridos.")

601 Registros Inseridos.
71 registros inseridos.


In [61]:
# Desconectando do Banco de dados
mycursor.close()
mydb.close()

print('Loggout Sucess to:', mydb)

Loggout Sucess to: <mysql.connector.connection.MySQLConnection object at 0x0000028EFC026370>
